In [8]:
import torch
import time
import pandas as pd

# These were all generated by ChatGPT
def benchmark_svd_methods(matrix_sizes, device='cuda', num_trials=5):
    results = {}

    for size in matrix_sizes:
        m, n = size
        print(f"Benchmarking for matrix size: {m}x{n}")
        results[size] = {}

        # Generate random matrix
        A = torch.randn(m, n, device=device)

        # Method 1: Full SVD
        times = []
        for _ in range(num_trials):
            torch.cuda.synchronize()
            start = time.time()
            U, S, V = torch.linalg.svd(A, full_matrices=False)
            torch.cuda.synchronize()
            times.append(time.time() - start)
        results[size]['Full SVD'] = sum(times) / num_trials

        # Method 2: Singular values only
        times = []
        for _ in range(num_trials):
            torch.cuda.synchronize()
            start = time.time()
            S = torch.linalg.svdvals(A)
            torch.cuda.synchronize()
            times.append(time.time() - start)
        results[size]['Singular Values Only'] = sum(times) / num_trials

        # Method 3: Randomized SVD (Low-rank Approximation)
        rank = min(m, n) // 2  # Approximate with half the full rank
        times = []
        for _ in range(num_trials):
            torch.cuda.synchronize()
            start = time.time()
            Q, _ = torch.linalg.qr(A @ torch.randn(n, rank, device=device))
            B = Q.T @ A
            U_hat, S, V = torch.linalg.svd(B, full_matrices=False)
            U = Q @ U_hat
            torch.cuda.synchronize()
            times.append(time.time() - start)
        results[size]['Randomized SVD'] = sum(times) / num_trials

        # Method 4: Power Iteration for top singular value
        num_iters = 10
        times = []
        for _ in range(num_trials):
            torch.cuda.synchronize()
            start = time.time()
            v = torch.randn(n, 1, device=device)
            for _ in range(num_iters):
                v = A.T @ (A @ v)
                v /= torch.norm(v)
            sigma = torch.norm(A @ v)
            torch.cuda.synchronize()
            times.append(time.time() - start)
        results[size]['Power Iteration'] = sum(times) / num_trials

        # Method 5: Lanczos using torch.linalg.eigh (for symmetric case)
        if m == n:  # Only for square matrices
            times = []
            for _ in range(num_trials):
                torch.cuda.synchronize()
                start = time.time()
                S, _ = torch.linalg.eigh(A.T @ A)
                S = torch.sqrt(torch.abs(S))
                torch.cuda.synchronize()
                times.append(time.time() - start)
            results[size]['Lanczos (Eigh)'] = sum(times) / num_trials

        # Method 6: Nyström Approximation
        num_samples = min(m, n) // 4  # Select a quarter of the rows/columns
        times = []
        for _ in range(num_trials):
            torch.cuda.synchronize()
            start = time.time()
            idx = torch.randint(0, m, (num_samples,), device=device)
            C = A[idx, :]
            W = C @ C.T
            S, U = torch.linalg.eigh(W)
            S = torch.sqrt(torch.abs(S))
            torch.cuda.synchronize()
            times.append(time.time() - start)
        results[size]['Nyström Approximation'] = sum(times) / num_trials

        # Method 7: Chebyshev Polynomial Approximation
        poly_order = 10  # Degree of the polynomial
        times = []
        for _ in range(num_trials):
            torch.cuda.synchronize()
            start = time.time()
            B = A.T @ A
            S = torch.zeros(min(m, n), device=device)
            x = torch.randn(n, device=device)
            for _ in range(poly_order):
                x = B @ x
                x /= torch.norm(x)
            S = torch.norm(B @ x)
            torch.cuda.synchronize()
            times.append(time.time() - start)
        results[size]['Chebyshev Approximation'] = sum(times) / num_trials

    return results

# Define matrix sizes to benchmark
matrix_sizes = [
    (512, 512),  # Small
    (1024, 1024),  # Medium
    (4096, 4096)   # Large
]

# Run benchmarks
results = benchmark_svd_methods(matrix_sizes)

# Print results
df = pd.DataFrame(results).T
print(df)


Benchmarking for matrix size: 512x512
Benchmarking for matrix size: 1024x1024
Benchmarking for matrix size: 4096x4096
           Full SVD  Singular Values Only  Randomized SVD  Power Iteration  \
512  512   0.021222              0.020927        0.009044         0.000840   
1024 1024  0.076803              0.066727        0.024412         0.000804   
4096 4096  7.624762              5.316650        0.658337         0.005815   

           Lanczos (Eigh)  Nyström Approximation  Chebyshev Approximation  
512  512         0.015751               0.018176                 0.002679  
1024 1024        0.012201               0.005668                 0.000661  
4096 4096        0.326542               0.015298                 0.011208  
